# Model Context Protocol (MCP) Labs - PART 1

## Overview
Production-ready MCP integration with **real Gmail and Calendar setup**:

### Part 1 Labs:
- **Lab 0**: Complete MCP Setup for Gmail & Google Calendar
- **Lab 1**: MCP Basics - Understanding Architecture
- **Lab 2**: Filesystem MCP - Document Management
- **Lab 3**: Gmail MCP - Real Email Workflows (cygenaidemo@gmail.com)
- **Lab 4**: Google Calendar MCP - Real Scheduling

### Use Cases
1. **HR Recruitment** - Interview invitations, scheduling
2. **AirBnb Management** - Booking confirmations, calendar blocking

### Sender Email
**cygenaidemo@gmail.com** - Pre-configured sender account

---

# LAB 0: Complete MCP Setup for Gmail & Calendar

## Prerequisites
- Node.js installed (for MCP servers)
- Google Cloud account
- Python 3.11+

## Step-by-Step Setup

## Part A: Google Cloud Project Setup

### Step 1: Create Google Cloud Project

1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Click **"Select a project"** → **"New Project"**
3. Project Name: `mcp-labs-project`
4. Click **"Create"**

### Step 2: Enable Required APIs

1. In Google Cloud Console, go to **"APIs & Services"** → **"Library"**
2. Search and enable:
   - **Gmail API**
   - **Google Calendar API**

### Step 3: Create OAuth 2.0 Credentials

1. Go to **"APIs & Services"** → **"Credentials"**
2. Click **"Create Credentials"** → **"OAuth client ID"**
3. If prompted, configure OAuth consent screen:
   - User Type: **External**
   - App name: `MCP Labs`
   - User support email: `cygenaidemo@gmail.com`
   - Developer email: `cygenaidemo@gmail.com`
   - Add scopes:
     - `https://www.googleapis.com/auth/gmail.send`
     - `https://www.googleapis.com/auth/gmail.readonly`
     - `https://www.googleapis.com/auth/calendar`
     - `https://www.googleapis.com/auth/calendar.events`
   - Add test user: `cygenaidemo@gmail.com`
   - Save and Continue

4. Back to **"Create OAuth client ID"**:
   - Application type: **Desktop app**
   - Name: `MCP Labs Desktop Client`
   - Click **"Create"**

5. **Download JSON** - Save as `credentials.json`

### Step 4: Place Credentials File

```bash
# Create directory structure
mkdir -p ~/.config/mcp

# Move downloaded credentials
mv ~/Downloads/credentials.json ~/.config/mcp/credentials.json
```

## Part B: Install MCP Servers

### Step 1: Install Node.js MCP Servers

Run these commands in your terminal:

```bash
# Install Gmail MCP Server
npm install -g @modelcontextprotocol/server-gmail

# Install Google Calendar MCP Server
npm install -g @modelcontextprotocol/server-google-calendar

# Install Filesystem MCP Server (optional)
npm install -g @modelcontextprotocol/server-filesystem

# Verify installations
which mcp-server-gmail
which mcp-server-google-calendar
```

### Step 2: Test MCP Server Connection

```bash
# Test Gmail MCP Server
npx @modelcontextprotocol/server-gmail --help

# Test Calendar MCP Server
npx @modelcontextprotocol/server-google-calendar --help
```

## Part C: Initial OAuth Authentication

### Step 1: Authenticate Gmail MCP Server

Run this command to authenticate (first time only):

```bash
# Start Gmail MCP server with credentials
GMAIL_CREDENTIALS=~/.config/mcp/credentials.json \
npx @modelcontextprotocol/server-gmail
```

**What happens:**
1. Browser opens with Google OAuth consent screen
2. Login with `cygenaidemo@gmail.com`
3. Click **"Allow"** to grant permissions
4. Token saved to `~/.config/mcp/gmail-token.json`
5. Press `Ctrl+C` to stop the server

### Step 2: Authenticate Google Calendar MCP Server

```bash
# Start Calendar MCP server with credentials
GOOGLE_CALENDAR_CREDENTIALS=~/.config/mcp/credentials.json \
npx @modelcontextprotocol/server-google-calendar
```

**What happens:**
1. Browser opens with Google OAuth consent screen
2. Login with `cygenaidemo@gmail.com`
3. Click **"Allow"** to grant calendar permissions
4. Token saved to `~/.config/mcp/calendar-token.json`
5. Press `Ctrl+C` to stop the server

### Step 3: Verify Tokens Created

```bash
ls -la ~/.config/mcp/

# You should see:
# credentials.json
# gmail-token.json
# calendar-token.json
```

## Part D: Create MCP Configuration File

### Create `mcp_config.json`

Save this file in your project directory:

```json
{
  "mcpServers": {
    "gmail": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-gmail"],
      "transport": "stdio",
      "env": {
        "GMAIL_CREDENTIALS": "~/.config/mcp/credentials.json",
        "GMAIL_TOKEN": "~/.config/mcp/gmail-token.json",
        "GMAIL_USER": "cygenaidemo@gmail.com"
      }
    },
    "google_calendar": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-google-calendar"],
      "transport": "stdio",
      "env": {
        "GOOGLE_CALENDAR_CREDENTIALS": "~/.config/mcp/credentials.json",
        "GOOGLE_CALENDAR_TOKEN": "~/.config/mcp/calendar-token.json",
        "GOOGLE_CALENDAR_USER": "cygenaidemo@gmail.com"
      }
    },
    "filesystem": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-filesystem", "./data"],
      "transport": "stdio"
    }
  }
}
```

### Create Environment File

Create `.env` file:

```bash
OPENAI_API_KEY=your-openai-api-key-here
MCP_CONFIG_PATH=./mcp_config.json
SENDER_EMAIL=cygenaidemo@gmail.com
```

## Part E: Verification Checklist

Before proceeding, verify:

✅ Google Cloud project created

✅ Gmail API and Calendar API enabled

✅ OAuth credentials downloaded (`credentials.json`)

✅ MCP servers installed via npm

✅ Gmail authenticated (token created)

✅ Calendar authenticated (token created)

✅ `mcp_config.json` created

✅ `.env` file configured

### Quick Test

```bash
# Test Gmail MCP
GMAIL_CREDENTIALS=~/.config/mcp/credentials.json \
GMAIL_TOKEN=~/.config/mcp/gmail-token.json \
GMAIL_USER=cygenaidemo@gmail.com \
npx @modelcontextprotocol/server-gmail

# Should start without errors
# Press Ctrl+C to stop
```

---

## Install Python Packages

In [ ]:
# Install required packages
!pip install -q -U langchain-openai langgraph langchain-core python-dotenv
!pip install -q -U langchain-mcp-adapters mcp
!pip install -q -U google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q -U google-api-python-client

print("✅ Python packages installed")

In [ ]:
# Core imports
import os
import json
import asyncio
from pathlib import Path
from dotenv import load_dotenv
from typing import List, Dict, Any
from datetime import datetime, timedelta

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent

# MCP imports
try:
    from langchain_mcp_adapters.client import MultiServerMCPClient
    from mcp import ClientSession, StdioServerParameters
    MCP_AVAILABLE = True
except ImportError:
    MCP_AVAILABLE = False
    print("⚠️ MCP adapters not installed. Run: pip install langchain-mcp-adapters mcp")

# Load environment
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Set OPENAI_API_KEY in .env file")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Configuration
SENDER_EMAIL = os.getenv("SENDER_EMAIL", "cygenaidemo@gmail.com")
MCP_CONFIG_PATH = os.getenv("MCP_CONFIG_PATH", "./mcp_config.json")

print("✅ Setup complete!")
print(f"📧 Sender Email: {SENDER_EMAIL}")
print(f"🔧 MCP Available: {MCP_AVAILABLE}")

## Load MCP Configuration and Initialize Client

In [ ]:
# Load MCP configuration
def load_mcp_config():
    """Load MCP configuration from file."""
    config_path = Path(MCP_CONFIG_PATH)
    
    if not config_path.exists():
        print(f"❌ MCP config not found at {MCP_CONFIG_PATH}")
        print("📝 Create mcp_config.json as shown in Lab 0")
        return None
    
    with open(config_path) as f:
        config = json.load(f)
    
    # Expand ~ in paths
    for server_name, server_config in config.get("mcpServers", {}).items():
        if "env" in server_config:
            for key, value in server_config["env"].items():
                if isinstance(value, str) and value.startswith("~"):
                    server_config["env"][key] = str(Path(value).expanduser())
    
    return config

# Initialize MCP client
async def initialize_mcp_client():
    """Initialize MultiServerMCPClient."""
    if not MCP_AVAILABLE:
        print("⚠️ MCP not available, using simulated tools")
        return None
    
    config = load_mcp_config()
    if not config:
        return None
    
    try:
        client = MultiServerMCPClient(config)
        print("✅ MCP Client initialized")
        return client
    except Exception as e:
        print(f"❌ Failed to initialize MCP client: {e}")
        print("📝 Check your mcp_config.json and authentication")
        return None

# Try to load config
mcp_config = load_mcp_config()
if mcp_config:
    print("✅ MCP Configuration loaded")
    print(f"📦 Available servers: {list(mcp_config.get('mcpServers', {}).keys())}")
else:
    print("⚠️ Using simulated MCP tools for demo")

## Sample Data

In [ ]:
# HR Database
HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "priya.sharma@example.com",  # Replace with real email for testing
            "phone": "+91-98765-43210",
            "position": "Senior Backend Engineer",
            "status": "screening"
        }
    }
}

# AirBnb Database
AIRBNB_DATABASE = {
    "properties": {
        "PROP001": {
            "name": "Cozy Goa Beach Villa",
            "location": "Candolim, Goa",
            "price_per_night": 8500
        }
    },
    "bookings": {
        "BOOK001": {
            "property_id": "PROP001",
            "guest_name": "Sarah Johnson",
            "guest_email": "sarah@example.com",  # Replace with real email for testing
            "checkin": "2025-10-20",
            "checkout": "2025-10-25",
            "status": "confirmed"
        }
    }
}

SAMPLE_RESUME = """PRIYA SHARMA - Senior Software Engineer
6+ years Python/AWS experience, FastAPI, microservices..."""

print("✅ Sample data loaded")

---

# LAB 1: MCP Architecture Understanding

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           MODEL CONTEXT PROTOCOL (MCP) ARCHITECTURE          ║
╚══════════════════════════════════════════════════════════════╝

YOUR APPLICATION (LangChain + LangGraph)
         │
         │ langchain-mcp-adapters
         ▼
┌─────────────────────┐
│MultiServerMCPClient │  ◄─── Reads mcp_config.json
└─────────┬───────────┘
          │
    ┌─────┴─────┬─────────┬──────────┐
    │           │         │          │
    ▼           ▼         ▼          ▼
┌────────┐ ┌────────┐ ┌────────┐ ┌────────┐
│ Gmail  │ │Calendar│ │FileSys │ │Discord │
│  MCP   │ │  MCP   │ │  MCP   │ │  MCP   │
└───┬────┘ └───┬────┘ └───┬────┘ └───┬────┘
    │          │          │          │
    ▼          ▼          ▼          ▼
[Gmail API][GCal API][Local FS][Discord API]

TRANSPORT: stdio (subprocess communication)

KEY FEATURES:
✓ Standard protocol for tool integration
✓ Multiple servers run simultaneously
✓ OAuth handled by MCP servers
✓ Tools discovered automatically
""")

if mcp_config:
    print("\n📋 Your MCP Configuration:")
    print(json.dumps(mcp_config, indent=2))

print("\n✅ Lab 1 Complete!")

---

# LAB 2: Filesystem MCP

In [ ]:
# Filesystem tools (simulated for demo, but can connect to real MCP)

@tool
def read_file(file_path: str) -> str:
    """Read file content."""
    if "priya" in file_path.lower():
        return SAMPLE_RESUME
    return f"File: {file_path} content..."

@tool
def write_file(file_path: str, content: str) -> str:
    """Write content to file."""
    return f"✅ Written {len(content)} bytes to {file_path}"

@tool
def list_files(directory: str) -> str:
    """List files in directory."""
    files = {"./data/resumes": ["priya_sharma.pdf"], "./data/contracts": ["BOOK001.pdf"]}
    return json.dumps({"directory": directory, "files": files.get(directory, [])}, indent=2)

filesystem_tools = [read_file, write_file, list_files]
filesystem_agent = create_react_agent(llm, tools=filesystem_tools, state_modifier="Document management assistant")

print("✅ Filesystem Agent ready")

In [ ]:
# Test Filesystem
print("\n🧪 LAB 2: Filesystem MCP\n" + "="*80)

result = filesystem_agent.invoke({
    "messages": [HumanMessage(content="Read Priya's resume from ./data/resumes/priya_sharma.pdf and summarize.")]
})

for msg in result['messages']:
    if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
        print(f"\n🤖 {msg.content}")
        break

print("\n✅ Lab 2 Complete!")

---

# LAB 3: Gmail MCP - Real Email Integration

**Sender**: cygenaidemo@gmail.com

**Receiver**: Specified by user

In [ ]:
# Gmail MCP Tools - Real Implementation

async def get_gmail_tools():
    """Get real Gmail MCP tools."""
    if not MCP_AVAILABLE or not mcp_config:
        print("⚠️ Using simulated Gmail tools")
        return get_simulated_gmail_tools()
    
    try:
        client = await initialize_mcp_client()
        if not client:
            return get_simulated_gmail_tools()
        
        # Get tools from Gmail MCP server
        gmail_tools = await client.get_tools(server_name="gmail")
        print(f"✅ Real Gmail MCP tools loaded: {[t.name for t in gmail_tools]}")
        return gmail_tools
    except Exception as e:
        print(f"⚠️ Failed to load real Gmail tools: {e}")
        return get_simulated_gmail_tools()

def get_simulated_gmail_tools():
    """Simulated Gmail tools for demo."""
    
    @tool
    def send_email(to: str, subject: str, body: str, cc: str = "") -> str:
        """
        Send email via Gmail.
        
        Args:
            to: Recipient email
            subject: Email subject
            body: Email body (supports HTML)
            cc: CC recipients (optional)
        """
        print(f"""
📧 EMAIL SENT (SIMULATED)
───────────────────────────────────────
From: {SENDER_EMAIL}
To: {to}
{f'CC: {cc}' if cc else ''}
Subject: {subject}
───────────────────────────────────────
{body[:300]}{'...' if len(body) > 300 else ''}
───────────────────────────────────────

⚠️ TO SEND REAL EMAILS:
1. Complete Lab 0 setup
2. Authenticate with cygenaidemo@gmail.com
3. MCP client will send actual emails
""")
        return f"✅ Email sent to {to}"
    
    @tool
    def search_emails(query: str, max_results: int = 10) -> str:
        """Search emails in Gmail."""
        return json.dumps([{"from": "example@email.com", "subject": "Test", "snippet": "..." }], indent=2)
    
    return [send_email, search_emails]

# Get Gmail tools (real or simulated)
try:
    gmail_tools = await get_gmail_tools()
except:
    gmail_tools = get_simulated_gmail_tools()

gmail_agent = create_react_agent(
    llm,
    tools=gmail_tools,
    state_modifier=f"""
You are an Email Assistant.
Sender email: {SENDER_EMAIL}
Write professional, personalized emails for HR and AirBnb use cases.
"""
)

print("✅ Gmail Agent ready")

In [ ]:
# Test Gmail - HR Interview Invitation
print("\n🧪 LAB 3: Gmail MCP\n" + "="*80)

# IMPORTANT: Replace with real email address for testing
test_recipient = "your-email@example.com"  # ← Change this!

print(f"\n--- Test 1: Send Interview Invitation ---")
print(f"📧 From: {SENDER_EMAIL}")
print(f"📧 To: {test_recipient}")
print()

result1 = gmail_agent.invoke({
    "messages": [HumanMessage(content=f"""
Send interview invitation email to {test_recipient}.

Details:
- Candidate: Priya Sharma
- Position: Senior Backend Engineer
- Interview Date: October 15, 2025 at 10:00 AM IST
- Interviewers: Rahul Verma (Engineering Manager), Vikram Singh (Tech Lead)
- Duration: 90 minutes
- Format: Technical (60 min) + Behavioral (30 min)
- Meeting Link: https://meet.google.com/abc-defg-hij

Make it professional and welcoming. Include:
- Interview agenda
- What to prepare
- Contact information
""")]
})

print("\n✅ Lab 3 Test 1 Complete!")

In [ ]:
# Test Gmail - AirBnb Booking Confirmation
print("\n--- Test 2: Send Booking Confirmation ---")

test_guest_email = "guest@example.com"  # ← Change this for testing!

result2 = gmail_agent.invoke({
    "messages": [HumanMessage(content=f"""
Send booking confirmation email to {test_guest_email}.

Booking Details:
- Guest: Sarah Johnson
- Property: Cozy Goa Beach Villa
- Location: Candolim Beach, Goa
- Check-in: October 20, 2025 at 2:00 PM
- Check-out: October 25, 2025 at 11:00 AM
- Guests: 4 people
- Total Cost: ₹42,500 (5 nights × ₹8,500/night)

Include:
- Property amenities (WiFi, Pool, Kitchen, Beach Access)
- House rules
- WiFi password: GoaBeach2025
- Check-in instructions
- Emergency contact: +91-98765-00000

Make it warm and informative.
""")]
})

print("\n✅ Lab 3 Test 2 Complete!")

---

# LAB 4: Google Calendar MCP - Real Scheduling

In [ ]:
# Google Calendar MCP Tools - Real Implementation

async def get_calendar_tools():
    """Get real Calendar MCP tools."""
    if not MCP_AVAILABLE or not mcp_config:
        print("⚠️ Using simulated Calendar tools")
        return get_simulated_calendar_tools()
    
    try:
        client = await initialize_mcp_client()
        if not client:
            return get_simulated_calendar_tools()
        
        calendar_tools = await client.get_tools(server_name="google_calendar")
        print(f"✅ Real Calendar MCP tools: {[t.name for t in calendar_tools]}")
        return calendar_tools
    except Exception as e:
        print(f"⚠️ Failed to load real Calendar tools: {e}")
        return get_simulated_calendar_tools()

def get_simulated_calendar_tools():
    """Simulated Calendar tools."""
    
    @tool
    def create_calendar_event(
        summary: str,
        start_time: str,
        end_time: str,
        attendees: str,
        description: str = "",
        location: str = ""
    ) -> str:
        """
        Create Google Calendar event.
        
        Args:
            summary: Event title
            start_time: Start time (ISO: 2025-10-15T10:00:00+05:30)
            end_time: End time (ISO format)
            attendees: Comma-separated emails
            description: Event description
            location: Location or meeting link
        """
        print(f"""
📅 CALENDAR EVENT CREATED (SIMULATED)
───────────────────────────────────────
Event: {summary}
Start: {start_time}
End: {end_time}
Attendees: {attendees}
Location: {location}
───────────────────────────────────────
{description}
───────────────────────────────────────

⚠️ TO CREATE REAL EVENTS:
Complete Lab 0 setup and authenticate.
""")
        return f"✅ Event '{summary}' created"
    
    @tool
    def check_availability(calendar_id: str, start_date: str, end_date: str) -> str:
        """Check calendar availability."""
        return json.dumps({"available_slots": [{"date": "2025-10-15", "time": "10:00-11:30"}]}, indent=2)
    
    @tool
    def block_calendar_dates(calendar_id: str, start_date: str, end_date: str, reason: str) -> str:
        """Block calendar dates."""
        return f"✅ Blocked {calendar_id} from {start_date} to {end_date}"
    
    return [create_calendar_event, check_availability, block_calendar_dates]

# Get Calendar tools
try:
    calendar_tools = await get_calendar_tools()
except:
    calendar_tools = get_simulated_calendar_tools()

calendar_agent = create_react_agent(
    llm,
    tools=calendar_tools,
    state_modifier="You are a Calendar & Scheduling Assistant. Check availability before creating events."
)

print("✅ Calendar Agent ready")

In [ ]:
# Test Calendar - Schedule Interview
print("\n🧪 LAB 4: Google Calendar MCP\n" + "="*80)

print("\n--- Test 1: Schedule Technical Interview ---")

result1 = calendar_agent.invoke({
    "messages": [HumanMessage(content="""
Schedule technical interview:

Steps:
1. Check availability for rahul.verma@techcorp.in on Oct 15, 2025
2. Create calendar event:
   - Title: Technical Interview - Priya Sharma - Senior Backend Engineer
   - Date: October 15, 2025
   - Time: 10:00 AM - 11:30 AM IST (UTC+5:30)
   - Duration: 90 minutes
   - Attendees: priya.sharma@example.com, rahul.verma@techcorp.in, vikram.singh@techcorp.in
   - Location: https://meet.google.com/abc-defg-hij
   - Description:
     * Technical round (60 min): Python, AWS, System Design
     * Behavioral round (30 min): Leadership, team collaboration
     * Please review candidate's GitHub: github.com/priya-sharma
""")]
})

print("\n✅ Lab 4 Test 1 Complete!")

In [ ]:
# Test Calendar - Block Property Dates
print("\n--- Test 2: Block Property Calendar for Booking ---")

result2 = calendar_agent.invoke({
    "messages": [HumanMessage(content="""
Block property calendar for confirmed booking:

- Property Calendar: goabeachvilla@property-calendars.com
- Start Date: 2025-10-20
- End Date: 2025-10-25
- Reason: BOOKING BOOK001 - Sarah Johnson - 4 guests
- Create all-day blocking event
""")]
})

print("\n✅ Lab 4 Test 2 Complete!")

---

# 🎯 Part 1 Summary

## Completed:

✅ **Lab 0**: Complete MCP setup with real Gmail & Calendar authentication

✅ **Lab 1**: MCP Architecture understanding

✅ **Lab 2**: Filesystem MCP for document management

✅ **Lab 3**: Gmail MCP with **cygenaidemo@gmail.com** as sender

✅ **Lab 4**: Google Calendar MCP for scheduling

## Real vs Simulated:

**Current Mode**: Using simulated tools (fallback mode)

**To Enable Real MCP**:
1. Complete all steps in Lab 0
2. Authenticate with cygenaidemo@gmail.com
3. Verify tokens created
4. MCP client will automatically use real APIs

## Next Steps:

Continue to **Part 2** for:
- Lab 5: Discord MCP
- Lab 6: GitHub MCP
- Lab 7: Custom MCP Servers
- Lab 8: Complete Multi-MCP Orchestration

## Configuration Files:

**mcp_config.json** - Server configuration

**.env** - Environment variables

**~/.config/mcp/** - Credentials and tokens